In [920]:

import pandas as pd
import pulp
data = pd.read_csv('Datafest2024_Test_FINAL.csv',sep= ";") 
# fecha_transaccion = data['fecha_transaccion']  
# codigo_cajero = data['codigo_cajero']  
# tipo_cajero = data['tipo_cajero']  
# demanda = data['demanda'] 
# demanda.loc
print(data)

       fecha_transaccion  codigo_cajero tipo_cajero  saldo_inicial  demanda  \
0               20240415              1           B        17610.0    18020   
1               20240415              2           B        19710.0     6560   
2               20240415              3           B        32670.0    24010   
3               20240415              4           B         1980.0    12910   
4               20240415              5           B         2930.0     6750   
...                  ...            ...         ...            ...      ...   
26196           20240527            696           A            NaN    31793   
26197           20240527            697           A            NaN    14334   
26198           20240527            698           A            NaN    30571   
26199           20240527            699           A            NaN    79193   
26200           20240527            700           A            NaN    27706   

       abastecimiento  saldo_final  
0            2

In [921]:

ultima_fecha=20240520
def crear_id_fecha(fecha):
    if fecha==ultima_fecha:
        return 0
    elif fecha==ultima_fecha+1:
        return 1
    elif fecha== ultima_fecha+2:
        return 2
    elif fecha==ultima_fecha+3:
        return 3
    elif fecha== ultima_fecha+4:
        return 4
    elif fecha==ultima_fecha+5:
        return 5
    elif fecha== ultima_fecha+6:
        return 6
    elif fecha== ultima_fecha+7:
        return 7
    else:
        return 999
    
def crear_fecha_id(id):
    if id==0:
        return ultima_fecha
    elif id==1:
        return ultima_fecha + 1
    elif id== 2:
        return ultima_fecha + 2
    elif id==3:
        return ultima_fecha + 3
    elif id== 4:
        return ultima_fecha + 4
    elif id==5:
        return ultima_fecha + 5
    elif id== 6:
        return ultima_fecha + 6
    elif id== 7:
        return ultima_fecha + 7
    else:
        return 999
data=data.sort_values(by=['fecha_transaccion','codigo_cajero'])

data.reset_index(drop=True, inplace=True)

data['id_fecha'] = data['fecha_transaccion'].apply(lambda x: crear_id_fecha(x))
# print(base4)
data.head()


,fecha_transaccion,codigo_cajero,tipo_cajero,saldo_inicial,demanda,abastecimiento,saldo_final,id_fecha
0,20240415,1,B,17610.0,18020,279240.0,278830.0,999
1,20240415,2,B,19710.0,6560,345800.0,358950.0,999
2,20240415,3,B,32670.0,24010,421390.0,430050.0,999
3,20240415,4,B,1980.0,12910,319210.0,308280.0,999
4,20240415,5,B,2930.0,6750,258190.0,254370.0,999


In [922]:

cajeros = list(range(1,700+1)) #codigo_cajero
dias = list(range(1,7+1)) #
#CAJEROS B -- 1     301  
#CAJEROS A -- 302   700

# Modelo de optimización
modelo = pulp.LpProblem("Optimización_Abastecimiento_ATM", pulp.LpMinimize)

costoA=0.001
costoB=0.0015

#variables 
B = pulp.LpVariable.dicts("Monto_Efectivo_B", [(i, j)  for i in cajeros if 1 <= i <= 301 for j in dias], lowBound=0, cat="Continuous")
A = pulp.LpVariable.dicts("Monto_Efectivo_A", [(i, j)  for i in cajeros if 302 <= i <= 700 for j in dias], lowBound=0, cat="Continuous")

#binarias
# binB=pulp.LpVariable.dicts("Monto_Efectivo_B", [(i, j)  for i in cajeros if 1 <= i <= 301 for j in dias], 0,1, cat="Binary")
# binA=pulp.LpVariable.dicts("Monto_Efectivo_B", [(i, j)  for i in cajeros if 302 <= i <= 700  for j in dias], 0,1, cat="Binary")



In [923]:


modelo += pulp.lpSum(costoA  * A[(i, j)] for i in cajeros if 302 <= i <= 700 for j in dias) +  (pulp.lpSum(costoB * B[(i, j)] for i in cajeros if 1 <= i <= 301 for j in dias)), "Costo_Total"
#  [binA[(i,j)] 
# [binB[(i,j)] 

In [924]:
#DIAS DE ABASTECIMIENTO
# DIAS DE SEMANA  - A
for i in cajeros: 
    if i in range(302, 701):
        for j in dias:
            if j==(3-1) or j==(4-1) or j==(6-1) or j==(7-1):
                modelo += A[(i,j)] == 0



In [925]:
# DIAS DE SEMANA  - B
for i in cajeros: 
    if i in range(1, 302):
        for j in dias:
            if j==(2-1) or j==(5-1) or j==(6-1) or j==(7-1):
                modelo += B[(i,j)] == 0

In [926]:
#Acumulacion de dias - A finde
for i in cajeros: 
    if i in range(302, 701):
        modelo += A[(i,5-1)] >= pulp.lpSum([data.loc[(data['codigo_cajero'] == i) & (data['id_fecha']==j),'demanda']] for j in range (5-1,7))

In [927]:
#Acumulacion de dias - B finde

for i in cajeros: 
    if i in range(1, 302):
        modelo += B[(i,4-1)] >= pulp.lpSum([data.loc[(data['codigo_cajero'] == i) & (data['id_fecha']==j),'demanda']] for j in range (4-1,7))

In [928]:
#Acumulacion de dias - A diasemana

for i in cajeros: 
    if i in range(302, 701):
        modelo += A[(i,2-1)] >= pulp.lpSum([data.loc[(data['codigo_cajero'] == i) & (data['id_fecha']==j),'demanda']] for j in range (2-1,4))

In [929]:
#Acumulacion de dias - B diasemana 

for i in cajeros: 
    if i in range(1, 302):
        modelo += B[(i,7)] >= pulp.lpSum([data.loc[(data['codigo_cajero'] == i) & (data['id_fecha']==j),'demanda']] for j in range (7))

In [930]:
#No acumulacion A

for i in cajeros: 
    if i in range(302, 701):
        for j in dias:
            if j==7:
                modelo += A[(i,j)] >= pulp.lpSum([data.loc[(data['codigo_cajero'] == i) & (data['id_fecha']==7),'demanda']] for j in [7])

In [931]:
#No acumulacion B

for i in cajeros: 
    if i in range(1, 302):
        for j in dias:
             if j==2:
                 modelo += B[(i,j)] >= [data.loc[(data['codigo_cajero'] == i) & (data['id_fecha']==j),'demanda']] 

In [932]:
# data2 = pd.read_csv('Datafest2024_Test.csv',sep= ",")  
# ultima_fecha = data2.loc[data2['saldo_final'].notna(), 'fecha_transaccion'].max()


# print(ultima_fecha)

# base=data2.loc[data2['fecha_transaccion']==ultima_fecha,['fecha_transaccion','codigo_cajero','tipo_cajero','saldo_final','demanda']].sort_values(by='codigo_cajero')

# base.reset_index(drop=True, inplace=True)

# base['id_fecha'] = base['fecha_transaccion'].apply(lambda x: crear_id_fecha(x))
# # print(base)
# base.head()


In [933]:
# modelo.solve()
# print(f"ES: {pulp.LpStatus[modelo.status]}")
# print(f"CC: {pulp.value(modelo.objective)}")

In [934]:
#CAPACIDAD A
for i in cajeros: 
    if i in range(302, 701):
        for j in dias:
            if j==1:

                demanda = float(data.loc[(data['codigo_cajero'] == i) & (data['id_fecha']==j),'demanda'])
                var = A[(i,j)]
                saldo_ini = float(data.loc[(data['codigo_cajero']==i) & (data['id_fecha']==j-1),'saldo_final'])
                

                suma=saldo_ini+var-demanda
                
                modelo += suma <= 1000000
                modelo += suma >= 200001
                index= data.loc[(data['codigo_cajero']==i) & (data['id_fecha']==j),'saldo_final'].index
                data.loc[index, 'saldo_final']= suma
           
                
        

C:\Users\Diana\AppData\Local\Temp\ipykernel_23212\2454731117.py:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  demanda = float(data.loc[(data['codigo_cajero'] == i) & (data['id_fecha']==j),'demanda'])
C:\Users\Diana\AppData\Local\Temp\ipykernel_23212\2454731117.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  saldo_ini = float(data.loc[(data['codigo_cajero']==i) & (data['id_fecha']==j-1),'saldo_final'])
C:\Users\Diana\AppData\Local\Temp\ipykernel_23212\2454731117.py:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  demanda = float(data.loc[(data['codigo_cajero'] == i) & (data['id_fecha']==j),'demanda'])
C:\Users\Diana\AppData\Local\Temp\ipykernel_23212\2454731117.py:9: FutureWarning: Callin

In [935]:
#CAPACIDAD B
for i in cajeros: 
    if i in range(1,302):
        for j in dias:
            if j==1:
                demanda = float(data.loc[(data['codigo_cajero'] == i) & (data['id_fecha']==j),'demanda'])
                var = B[(i,j)]
                saldo_ini = float(data.loc[(data['codigo_cajero']==i) & (data['id_fecha']==j-1),'saldo_final'])
                

                suma=saldo_ini+var-demanda
                
                modelo += suma <= 1300000
                modelo += suma >= 260001
                index= data.loc[(data['codigo_cajero']==i) & (data['id_fecha']==j),'saldo_final'].index
                data.loc[index, 'saldo_final']= suma


                

C:\Users\Diana\AppData\Local\Temp\ipykernel_23212\2198969330.py:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  demanda = float(data.loc[(data['codigo_cajero'] == i) & (data['id_fecha']==j),'demanda'])
C:\Users\Diana\AppData\Local\Temp\ipykernel_23212\2198969330.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  saldo_ini = float(data.loc[(data['codigo_cajero']==i) & (data['id_fecha']==j-1),'saldo_final'])
C:\Users\Diana\AppData\Local\Temp\ipykernel_23212\2198969330.py:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  demanda = float(data.loc[(data['codigo_cajero'] == i) & (data['id_fecha']==j),'demanda'])
C:\Users\Diana\AppData\Local\Temp\ipykernel_23212\2198969330.py:8: FutureWarning: Callin

In [936]:
# # Resolver el modelo
modelo.solve()

# # Resultados
print(f"ES: {pulp.LpStatus[modelo.status]}")
print(f"CC: {pulp.value(modelo.objective)}")


ES: Infeasible
CC: 1463523.2439999967


In [937]:
# # Resolver el modelo
modelo.solve()

# # Resultados
print(f"ES: {pulp.LpStatus[modelo.status]}")
print(f"CC: {pulp.value(modelo.objective)}") 

ES: Infeasible
CC: 1463523.2439999967


In [938]:
type(cajeros)

list

In [939]:
type(dias)

list

In [940]:
print(A[(i, j)].varValue)

0.0


In [941]:
A[(302, 1)]

Monto_Efectivo_A_(302,_1)

In [942]:
for var in modelo.variables():
    print(f'{var.name} = {var.varValue}')

Monto_Efectivo_A_(302,_1) = 708535.0
Monto_Efectivo_A_(302,_2) = 0.0
Monto_Efectivo_A_(302,_3) = 0.0
Monto_Efectivo_A_(302,_4) = 402240.0
Monto_Efectivo_A_(302,_5) = 0.0
Monto_Efectivo_A_(302,_6) = 0.0
Monto_Efectivo_A_(302,_7) = 0.0
Monto_Efectivo_A_(303,_1) = 702895.0
Monto_Efectivo_A_(303,_2) = 0.0
Monto_Efectivo_A_(303,_3) = 0.0
Monto_Efectivo_A_(303,_4) = 354171.0
Monto_Efectivo_A_(303,_5) = 0.0
Monto_Efectivo_A_(303,_6) = 0.0
Monto_Efectivo_A_(303,_7) = 0.0
Monto_Efectivo_A_(304,_1) = 686620.0
Monto_Efectivo_A_(304,_2) = 0.0
Monto_Efectivo_A_(304,_3) = 0.0
Monto_Efectivo_A_(304,_4) = 333319.0
Monto_Efectivo_A_(304,_5) = 0.0
Monto_Efectivo_A_(304,_6) = 0.0
Monto_Efectivo_A_(304,_7) = 0.0
Monto_Efectivo_A_(305,_1) = 730335.0
Monto_Efectivo_A_(305,_2) = 0.0
Monto_Efectivo_A_(305,_3) = 0.0
Monto_Efectivo_A_(305,_4) = 398193.0
Monto_Efectivo_A_(305,_5) = 0.0
Monto_Efectivo_A_(305,_6) = 0.0
Monto_Efectivo_A_(305,_7) = 0.0
Monto_Efectivo_A_(306,_1) = 716131.0
Monto_Efectivo_A_(306,_2) =

In [948]:
for i in cajeros:
    if i in range(1, 302):
        for j in dias:
            print(f"Efectivo abastecido en {i} el {j}: {B[(i, j)].varValue}")

for i in cajeros:
    if i in range(302, 701):
        for j in dias:
            print(f"Efectivo abastecido en {i} el {j}: {A[(i, j)].varValue}")
            # print(data.loc[(data['codigo_cajero']==i) & (data['id_fecha']==j),['saldo_final','demanda'])



Efectivo abastecido en 1 el 1: 0.0
Efectivo abastecido en 1 el 2: 363801.0
Efectivo abastecido en 1 el 3: 724960.0
Efectivo abastecido en 1 el 4: 0.0
Efectivo abastecido en 1 el 5: 0.0
Efectivo abastecido en 1 el 6: 0.0
Efectivo abastecido en 1 el 7: 1151054.0
Efectivo abastecido en 2 el 1: 0.0
Efectivo abastecido en 2 el 2: 325548.0
Efectivo abastecido en 2 el 3: 606912.0
Efectivo abastecido en 2 el 4: 0.0
Efectivo abastecido en 2 el 5: 0.0
Efectivo abastecido en 2 el 6: 0.0
Efectivo abastecido en 2 el 7: 949657.0
Efectivo abastecido en 3 el 1: 0.0
Efectivo abastecido en 3 el 2: 389943.0
Efectivo abastecido en 3 el 3: 882176.0
Efectivo abastecido en 3 el 4: 0.0
Efectivo abastecido en 3 el 5: 0.0
Efectivo abastecido en 3 el 6: 0.0
Efectivo abastecido en 3 el 7: 1310959.0
Efectivo abastecido en 4 el 1: 0.0
Efectivo abastecido en 4 el 2: 335991.0
Efectivo abastecido en 4 el 3: 918731.0
Efectivo abastecido en 4 el 4: 0.0
Efectivo abastecido en 4 el 5: 0.0
Efectivo abastecido en 4 el 6: 0.

In [944]:
for i in cajeros:
    if i in range(302, 701):
        for j in dias:
            print(f"Efectivo abastecido en {i} el {j}: {A[(i, j)].varValue}")

for i in cajeros:
    if i in range(1, 302):
        for j in dias:
            print(f"Efectivo abastecido en {i} el {j}: {B[(i, j)].varValue}")

Efectivo abastecido en 302 el 1: 708535.0
Efectivo abastecido en 302 el 2: 0.0
Efectivo abastecido en 302 el 3: 0.0
Efectivo abastecido en 302 el 4: 402240.0
Efectivo abastecido en 302 el 5: 0.0
Efectivo abastecido en 302 el 6: 0.0
Efectivo abastecido en 302 el 7: 0.0
Efectivo abastecido en 303 el 1: 702895.0
Efectivo abastecido en 303 el 2: 0.0
Efectivo abastecido en 303 el 3: 0.0
Efectivo abastecido en 303 el 4: 354171.0
Efectivo abastecido en 303 el 5: 0.0
Efectivo abastecido en 303 el 6: 0.0
Efectivo abastecido en 303 el 7: 0.0
Efectivo abastecido en 304 el 1: 686620.0
Efectivo abastecido en 304 el 2: 0.0
Efectivo abastecido en 304 el 3: 0.0
Efectivo abastecido en 304 el 4: 333319.0
Efectivo abastecido en 304 el 5: 0.0
Efectivo abastecido en 304 el 6: 0.0
Efectivo abastecido en 304 el 7: 0.0
Efectivo abastecido en 305 el 1: 730335.0
Efectivo abastecido en 305 el 2: 0.0
Efectivo abastecido en 305 el 3: 0.0
Efectivo abastecido en 305 el 4: 398193.0
Efectivo abastecido en 305 el 5: 0.

In [945]:
import csv

In [946]:
df = pd.DataFrame(columns=['cajero', 'dia', 'efectivo'])
for i in cajeros:
    if i in range(302, 701):
        for j in dias:
            df = df._append({'cajero': i, 'dia': j, 'efectivo': A[(i, j)].varValue}, ignore_index=True)

    elif i in range(1, 302):
        for j in dias:
            df = df._append({'cajero': i, 'dia': j, 'efectivo': B[(i, j)].varValue}, ignore_index=True)

C:\Users\Diana\AppData\Local\Temp\ipykernel_23212\2758067667.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = df._append({'cajero': i, 'dia': j, 'efectivo': B[(i, j)].varValue}, ignore_index=True)


In [947]:
df.to_csv('resultados_prueba.csv', index=False)